## Getting Started with prompt Template and Chat Prompt Template

### Use PromptTemplate to create a template for a string prompt.
By default, PromptTemplate uses Python’s str.format syntax for templating.

### Install the following libraries if not installed already

In [22]:
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-openai
# !pip install python-dotenv
# !pip install langchain-google-genai

### Import Libraries

In [2]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

### Use your model of Choice - OpenAI or Google's Gemini. 
We are using the ChatCompletion APIs of these LLMs

In [4]:
#model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0.5)
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro",  google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.5, convert_system_message_to_human=True)

### A simple string based Prompt formatting

In [5]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt = prompt_template.format(adjective="sad", content="British Weather")

response = model.invoke(prompt)
print(response)

c:\Users\diksh\.virtualenvs\rag-app-Nid9tYsN\Lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


content='An old man is sitting on a park bench in Manchester, staring up at the thick, grey clouds.\n\nA tourist sits down next to him and says cheerfully, "A bit miserable today, isn\'t it?"\n\nThe old man sighs, never taking his eyes off the sky. "Son," he says, "I\'ve lived here for 80 years. We don\'t have weather. We just have different types of disappointment."' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': []} id='run--d4a4a02b-645f-4818-8b54-0efe3d7fa5f6-0' usage_metadata={'input_tokens': 10, 'output_tokens': 91, 'total_tokens': 1520, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1419}}


### ChatPromptTemplate: The prompt to chat models is a list of chat messages.
Each chat message is associated with content, and an additional parameter called role. <br>
For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [6]:
from langchain_core.prompts import ChatPromptTemplate

In [7]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
prompt = chat_template.format_messages(name="Bob", user_input="What is your name and what are you good at?")
response = model.invoke(prompt)
print(response)

c:\Users\diksh\.virtualenvs\rag-app-Nid9tYsN\Lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


content="My name is Bob!\n\nAs a helpful AI, I'm good at a lot of things. You can think of me as a versatile assistant. For example, I can:\n\n*   **Answer your questions** on a huge range of topics.\n*   **Brainstorm ideas** with you for a project or story.\n*   **Write and edit text**, like emails, essays, or even poems.\n*   **Summarize** long articles or documents for you.\n*   **Explain complex subjects** in a simple way.\n*   **Help you with coding** and debugging.\n\nBasically, I'm here to help you learn, create, and get things done. What's on your mind?" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': []} id='run--f1e374ab-1daf-47be-a1f8-2e0645a98849-0' usage_metadata={'input_tokens': 41, 'output_tokens': 151, 'total_tokens': 1081, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 889}}


### Various ways of formatting System/Human/AI prompts

In [50]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """You are a helpful assistant that re-writes the user's text to 
                sound more upbeat."""
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
prompt = chat_template.format_messages(text="I don't like action movies")

response = model.invoke(prompt)
print(response)

content='I prefer other genres over action movies!'


### Providing a Context along with the Prompt

In [51]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In [52]:
print(model.invoke(prompt))

content="Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library."


### Langchain’s ```FewShotPromptTemplate``` caters to source knowledge input. The idea is to “train” the model on a few examples — we call this few-shot learning — and these examples are given to the model within the prompt.

The goal of few-shot prompt templates are to dynamically select examples based on an input, and then format the examples in a final prompt to provide for the model.<br>

<b>Fixed Examples</b><br>
The most basic (and common) few-shot prompting technique is to use a fixed prompt example. This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production. <br>

The basic components of the template are: - examples: A list of dictionary examples to include in the final prompt. - example_prompt: converts each example into 1 or more messages through its format_messages method. A common example would be to convert each example into one human message and one AI message response, or a human message followed by a function call message.

In [53]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, 
    {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

In [54]:
# create a prompt example from above template

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

In [55]:
# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [56]:
query = "What movie should I watch this evening?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What movie should I watch this evening?
AI: 


In [57]:
chain = few_shot_prompt_template | model

chain.invoke({"query": query})

AIMessage(content='How about "The Shawshank Redemption"? It\'s a classic that will make you question your own existence.')